<a href="https://colab.research.google.com/github/soumilhooda/MLDLNNtoCV/blob/main/Q5_NNFL_Assignment2_SoumilHooda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

In [2]:
from google.colab import files
uploaded = files.upload()

Saving data5.xlsx to data5.xlsx


In [3]:
full=pd.read_excel('data5.xlsx',header=None,names=["A","B","C","D","E","F","G","class"])
full.head()

,A,B,C,D,E,F,G,class
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1.0
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1.0
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1.0
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1.0
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1.0


In [4]:
full=full.values
np.random.shuffle(full)

x_full=full[:,0:7]
y_full=full[:,7:8]

In [5]:
x_normalized=np.ones((x_full.shape[0],x_full.shape[1]+1))
for i in range(x_full.shape[1]):
  x_normalized[:,i+1:i+2]=(x_full[:,i:i+1]-np.amin(x_full[:,i:i+1]))/(np.amax(x_full[:,i:i+1])-np.amin(x_full[:,i:i+1]))

In [6]:
y_normalized=np.zeros((len(y_full),3))

for i in range(len(y_full)):
  if y_full[i][0]==1:
    y_normalized[i][0]=1
    y_normalized[i][1]=0
    y_normalized[i][2]=0
  elif y_full[i][0]==2:
    y_normalized[i][0]=0
    y_normalized[i][1]=1
    y_normalized[i][2]=0
  elif y_full[i][0]==3:
    y_normalized[i][0]=0
    y_normalized[i][1]=0
    y_normalized[i][2]=1

In [7]:
def divide_tr_va_te(x_normalized,y_normalized):
  x_tr=x_normalized[:math.floor(0.7*x_normalized.shape[0])]
  x_va=x_normalized[math.floor(0.7*x_normalized.shape[0]):math.floor(0.8*x_normalized.shape[0])]
  x_te=x_normalized[math.floor(0.8*x_normalized.shape[0]):]

  y_tr=y_normalized[:math.floor(0.7*y_normalized.shape[0])]
  y_va=y_normalized[math.floor(0.7*y_normalized.shape[0]):math.floor(0.8*y_normalized.shape[0])]
  y_te=y_normalized[math.floor(0.8*y_normalized.shape[0]):]
  return x_tr,x_va,x_te,y_tr,y_va,y_te

x_tr,x_va,x_te,y_tr,y_va,y_te=divide_tr_va_te(x_normalized,y_normalized)
print(x_tr.shape)
print(x_va.shape)
print(x_te.shape)
print(y_tr.shape)

(147, 8)
(21, 8)
(42, 8)
(147, 3)


In [8]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

In [9]:
def delta_sigm(x):
  return np.multiply(x,1-x)

In [10]:
def fpa(inp,w1,b1,w2,b2):
  h=inp@w1+b1
  h=sigmoid(h)
  o=h@w2+b2
  o=sigmoid(o)
  return inp,h,o

In [11]:
def bpa(inp,a1,a2,a3,w1,b1,w2,b2,alpha,lam):
  delta3=-1*((inp-a3)*delta_sigm(a3))
  delta2=np.multiply(delta_sigm(a2), (delta3 @ w2.T) )
  w2=w2*(1-alpha*lam)-(alpha)*(a2.T @ delta3)
  w1=w1*(1-alpha*lam)-(alpha)*(a1.T @ delta2)
  b2=b2-(alpha)*np.sum(delta3,axis=0)
  b1=b1-(alpha)*np.sum(delta2,axis=0)
  return w1,b1,w2,b2

In [12]:
def cost(w1,b1,w2,b2,inp,a1,a2,a3,lam,hidden):
  err_sq=((a3-inp)**2)/2
  err=np.sum(np.sum(err_sq))/len(inp)
  pj=np.sum(a2,axis=1)/len(inp)
  err2=(lam/2) * (np.sum(np.sum(w1**2))+np.sum(np.sum(w2**2)))
  j=err+err2
  return j

In [13]:
def error(pred,act):
  return np.sum((pred-act)**2)/len(pred)

In [14]:
def pre_training(x_tr,x_va,hidden_least,hidden_max):
  hidden_arr=np.arange(hidden_least,hidden_max+1,1)

  alpha_arr=np.arange(0.01,0.1,0.01)
  lam_arr=np.arange(0.1,1,0.1)

  j_min=float('inf')
  alpha_min=0
  lam_min=0
  hidden_min=0
  w1_min=0
  w2_min=0
  b1_min=0
  b2_min=0

  for hidden_index in range(len(hidden_arr)):
    for alpha_index in range(len(alpha_arr)):
      for lam_index in range(len(lam_arr)):
        hidden=hidden_arr[hidden_index]
        alpha=alpha_arr[alpha_index]
        lam=lam_arr[lam_index]

        inp=x_tr

        w1=np.zeros((inp.shape[1],hidden))
        w2=np.zeros((hidden,inp.shape[1]))
        b1=np.zeros((1,hidden))
        b2=np.zeros((1,inp.shape[1]))

        for i in range(500):
          a1,a2,a3=fpa(inp,w1,b1,w2,b2)
          w1,b1,w2,b2=bpa(inp,a1,a2,a3,w1,b1,w2,b2,alpha,lam)

        a1,a2,a3=fpa(x_va,w1,b1,w2,b2)
        j=error(a3,x_va)
        if j<=j_min:
          j_min=j
          alpha_min=alpha
          lam_min=lam
          hidden_min=hidden
          w1_min=w1
          w2_min=w2
          b1_min=b1
          b2_min=b2
  print(f'hidden_min is {hidden_min}, j is {j_min}, alpha is {alpha_min}, lambda is {lam_min}')
  return w1_min,b1_min,w2_min,b2_min,hidden_min

In [15]:
w1,b1,w11,b11,hidden1=pre_training(x_tr,x_va,5,x_tr.shape[1]-1)

hidden_min is 7, j is 0.08913234313320365, alpha is 0.02, lambda is 0.1


In [16]:
a1_tr,a2_tr,a3_tr=fpa(x_tr,w1,b1,w11,b11)
a1_va,a2_va,a3_va=fpa(x_va,w1,b1,w11,b11)

In [17]:

w2,b2,w22,b22,hidden2=pre_training(a2_tr,a2_va,4,hidden1-1)
a21_tr,a22_tr,a23_tr=fpa(a2_tr,w2,b2,w22,b22)
a21_va,a22_va,a23_va=fpa(a2_va,w2,b2,w22,b22)

print('a22_tr ',a22_tr.shape)

hidden_min is 6, j is 0.0020477386684816952, alpha is 0.060000000000000005, lambda is 0.1
a22_tr  (147, 6)


In [18]:
w3,b3,w33,b33,hidden3=pre_training(a22_tr,a22_va,3,hidden2-1)

hidden_min is 5, j is 0.00183306325600429, alpha is 0.02, lambda is 0.1


In [19]:
def fpa_stack(inp,w1,b1,w2,b2,w3,b3,w4,b4):
  h1=inp@w1+b1
  h1=sigmoid(h1)
  h2=h1@w2+b2
  h2=sigmoid(h2)
  h3=h2@w3+b3
  h3=sigmoid(h3)
  # print(h3.shape)
  h4=h3@w4+b4
  h4=sigmoid(h4)
  
  return inp,h1,h2,h3,h4

In [20]:
def bpa_stack(inp,a1,a2,a3,a4,a5,w1,b1,w2,b2,w3,b3,w4,b4,alpha,lam,y_tr):
  delta5=-1*(np.multiply(y_tr-a5,delta_sigm(a5)))
  delta4=np.multiply( (delta5 @ w4.T),delta_sigm(a4) )
  delta3=np.multiply( (delta4 @ w3.T),delta_sigm(a3))
  delta2=np.multiply( (delta3 @ w2.T),delta_sigm(a2))

  w4=w4*(1-alpha*lam)-(alpha)*(a4.T @ delta5)  
  w3=w3*(1-alpha*lam)-(alpha)*(a3.T @ delta4)
  w2=w2*(1-alpha*lam)-(alpha)*(a2.T @ delta3)
  w1=w1*(1-alpha*lam)-(alpha)*(a1.T @ delta2)

  b4=b4-(alpha)*np.sum(delta5,axis=0)
  b3=b3-(alpha)*np.sum(delta4,axis=0)
  b2=b2-(alpha)*np.sum(delta3,axis=0)
  b1=b1-(alpha)*np.sum(delta2,axis=0)
  return w1,b1,w2,b2,w3,b3,w4,b4

In [21]:
def cost_stack(w1,b1,w2,b2,w3,b3,w4,b4,inp,a5,lam,y_tr):
  err_sq=((a5-y_tr)**2)/2
  err=np.sum(np.sum(err_sq))/len(inp)
  err2=(lam/2) * (np.sum(np.sum(w1**2))+np.sum(np.sum(w2**2))+np.sum(np.sum(w3**2))+np.sum(np.sum(w4**2)))
  j=err+err2
  return j

In [22]:
def hot_decoding(m):
  temp=np.zeros((len(m),1))
  
  for i in range(len(m)):
    temp[i]=np.argmax(m[i])+1
  
  return temp

def pred_err(y_pred,y_te):
  y_pred_dec=hot_decoding(y_pred)
  y_te_dec=hot_decoding(y_te)

  err=0
  for i in range(len(y_te)):
    if y_pred_dec[i]!=y_te_dec[i]:
      err+=1
  return err

In [23]:
def stacked(x_tr,x_va,w1_opt,b1_opt,w2_opt,b2_opt,w3_opt,b3_opt):
  alpha_arr=np.arange(0.01,0.1,0.01)
  lam_arr=np.arange(0.1,1,0.1)

  j_min=float('inf')
  alpha_min=0
  lam_min=0
  w1_min=0
  w2_min=0
  w3_min=0
  w4_min=0

  b1_min=0
  b2_min=0
  b3_min=0
  b4_min=0

  for alpha_index in range(len(alpha_arr)):
    for lam_index in range(len(lam_arr)):

        w1=w1_opt
        b1=b1_opt
        w2=w2_opt
        b2=b2_opt
        w3=w3_opt
        b3=b3_opt

        alpha=alpha_arr[alpha_index]
        lam=lam_arr[lam_index]

        inp=x_tr

        w4=np.zeros((hidden3,3))
        b4=np.zeros((1,3))

        for i in range(500):
          a1,a2,a3,a4,a5=fpa_stack(inp,w1,b1,w2,b2,w3,b3,w4,b4)
          w1,b1,w2,b2,w3,b3,w4,b4=bpa_stack(inp,a1,a2,a3,a4,a5,w1,b1,w2,b2,w3,b3,w4,b4,alpha,lam,y_tr)

        a1,a2,a3,a4,a5=fpa_stack(x_va,w1,b1,w2,b2,w3,b3,w4,b4)
        j=pred_err(a5,y_va)
        if j<=j_min:
          j_min=j
          alpha_min=alpha
          lam_min=lam
          w1_min=w1
          w2_min=w2
          w3_min=w3
          w4_min=w4

          b1_min=b1
          b2_min=b2
          b3_min=b3
          b4_min=b4
  print(f'j is {j_min}, alpha is {alpha_min}, lambda is {lam_min}')
  return w1_min,b1_min,w2_min,b2_min,w3_min,b3_min,w4_min,b4_min

In [24]:
w1_stack,b1_stack,w2_stack,b2_stack,w3_stack,b3_stack,w4_stack,b4_stack=stacked(x_tr,x_va,w1,b1,w2,b2,w3,b3)

j is 3, alpha is 0.03, lambda is 0.2


In [27]:
a1,a2,a3,a4,a5=fpa_stack(x_te,w1_stack,b1_stack,w2_stack,b2_stack,w3_stack,b3_stack,w4_stack,b4_stack)

In [28]:
y_output_actual=hot_decoding(y_te)
y_output=hot_decoding(a5)

confusion_matrix=pd.crosstab(y_output_actual.flatten(),y_output.flatten())
confusion_matrix=np.asarray(confusion_matrix)
print("Confusion matrix is ")
print(confusion_matrix)
acc=(confusion_matrix[0][0]+confusion_matrix[1][1]+confusion_matrix[2][2])/len(x_te)*100
print("Overall accuracy is ",acc)
for i in range(3):
  acc=confusion_matrix[i][i]/(np.sum(confusion_matrix[i]))
  print(f"Accuracy of class {i+1} :",acc*100)

Confusion matrix is 
[[ 7  0  5]
 [ 0 15  0]
 [ 0  0 15]]
Overall accuracy is  88.09523809523809
Accuracy of class 1 : 58.333333333333336
Accuracy of class 2 : 100.0
Accuracy of class 3 : 100.0
